In [72]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical


#Read the CSV file
data = pd.read_csv('amazon_reviews.csv')
import nltk




In [73]:
import re
import nltk
import pandas as pd
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score


# Cleaning of the dataset
from nltk.corpus import stopwords

# Drop NULL values 
data = data.dropna() 

stop_words = set(stopwords.words('english'))


def preprocess_text(text):
        text = text.lower()  # Convert to lowercase
        text = re.sub(r'<.*?>', '', text) #Remove html brackets and s
        text = re.sub(r'\d+', '', text)  # Remove numbers
        text = re.sub(r'\s+', ' ', text).strip()  # Remove extra spaces
        text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
        words = word_tokenize(text)  # Tokenize
        filtered_words = [word for word in words if word not in stop_words]  # Remove stopwords
        return ' '.join(filtered_words)
   
#Apply preprocessing steps on the data
data['cleaned_review'] = data['cleaned_review'].apply(preprocess_text)

# Encode the labels
label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
data['sentiments'] = data['sentiments'].map(label_mapping)

#Encode review score
# 1,2 -> Negative (0)
data.loc[data['review_score'] < 3, 'review_score'] = 0

# 3 -> Neutral (2)
data.loc[data['review_score'] == 3, 'review_score'] = 1

# 4,5 -> Positive (1)
data.loc[data['review_score'] > 3, 'review_score'] = 2



In [74]:
data.head()

,sentiments,cleaned_review,cleaned_review_length,review_score
0,2,wish would gotten one earlier love makes worki...,19,2.0
1,1,learned lesson open package use product right ...,88,0.0
2,1,slow lags find better option,9,0.0
3,1,roller ball stopped working within months mini...,12,0.0
4,1,like color size days return period hold charge,21,0.0


In [75]:
import pandas as pd
import numpy as np
import re
import nltk
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'], data['sentiments'], test_size=0.2, random_state=42)

# Tokenization 
max_words = 10000
max_len = 200

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X_train)
sequences_train = tokenizer.texts_to_sequences(X_train)
sequences_test = tokenizer.texts_to_sequences(X_test)

# Padding
X_train_pad = pad_sequences(sequences_train, maxlen=max_len)
X_test_pad = pad_sequences(sequences_test, maxlen=max_len)

y_train_cat = to_categorical(y_train, num_classes=3)
y_test_cat = to_categorical(y_test, num_classes=3)


In [76]:
from keras.models import Sequential
from keras.layers import Embedding, SimpleRNN, Dense
from keras.callbacks import ModelCheckpoint

# Simple RNN model and training
rnn_model = Sequential()
rnn_model.add(Embedding(max_words, 128, input_length=max_len))
rnn_model.add(SimpleRNN(128))
rnn_model.add(Dense(3, activation='softmax'))

rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the RNN model
rnn_model_checkpoint = ModelCheckpoint('rnn_model.h5', monitor='val_accuracy', save_best_only=True)

rnn_history = rnn_model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_pad, y_test_cat), callbacks=[rnn_model_checkpoint])


Epoch 1/5
217/217 [==============================] - 17s 77ms/step - loss: 0.6206 - accuracy: 0.7370 - val_loss: 0.4623 - val_accuracy: 0.8356
Epoch 2/5
  2/217 [..............................] - ETA: 14s - loss: 0.3019 - accuracy: 0.9297

C:\Users\Lenovo\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


217/217 [==============================] - 15s 69ms/step - loss: 0.2855 - accuracy: 0.9026 - val_loss: 0.4075 - val_accuracy: 0.8636
Epoch 3/5
217/217 [==============================] - 14s 66ms/step - loss: 0.1235 - accuracy: 0.9636 - val_loss: 0.4554 - val_accuracy: 0.8558
Epoch 4/5
217/217 [==============================] - 15s 68ms/step - loss: 0.1072 - accuracy: 0.9658 - val_loss: 0.5222 - val_accuracy: 0.8653
Epoch 5/5
217/217 [==============================] - 14s 66ms/step - loss: 0.0461 - accuracy: 0.9886 - val_loss: 0.5589 - val_accuracy: 0.8578


In [77]:
from keras.layers import LSTM

# LSTM model and trining
lstm_model = Sequential()
lstm_model.add(Embedding(max_words, 128, input_length=max_len))
lstm_model.add(LSTM(128))
lstm_model.add(Dense(3, activation='softmax'))

lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the LSTM model
lstm_model_checkpoint = ModelCheckpoint('lstm_model.h5', monitor='val_accuracy', save_best_only=True)

lstm_history = lstm_model.fit(X_train_pad, y_train_cat, epochs=10, batch_size=64, validation_data=(X_test_pad, y_test_cat), callbacks=[lstm_model_checkpoint])


Epoch 1/5
217/217 [==============================] - 43s 195ms/step - loss: 0.5946 - accuracy: 0.7445 - val_loss: 0.4494 - val_accuracy: 0.8307
Epoch 2/5
217/217 [==============================] - 42s 192ms/step - loss: 0.3287 - accuracy: 0.8794 - val_loss: 0.3955 - val_accuracy: 0.8535
Epoch 3/5
217/217 [==============================] - 41s 190ms/step - loss: 0.2301 - accuracy: 0.9221 - val_loss: 0.4057 - val_accuracy: 0.8587
Epoch 4/5
217/217 [==============================] - 42s 193ms/step - loss: 0.1803 - accuracy: 0.9399 - val_loss: 0.4068 - val_accuracy: 0.8682
Epoch 5/5
217/217 [==============================] - 42s 192ms/step - loss: 0.1438 - accuracy: 0.9544 - val_loss: 0.4464 - val_accuracy: 0.8653


In [78]:
# Evaluate the models on validation data
rnn_model.load_weights('rnn_model.h5')
lstm_model.load_weights('lstm_model.h5')

rnn_val_accuracy = rnn_model.evaluate(X_test_pad, y_test_cat)[1]
lstm_val_accuracy = lstm_model.evaluate(X_test_pad, y_test_cat)[1]

print(f'RNN Validation Accuracy: {rnn_val_accuracy}')
print(f'LSTM Validation Accuracy: {lstm_val_accuracy}')


109/109 [==============================] - 1s 11ms/step - loss: 0.5222 - accuracy: 0.8653
RNN Validation Accuracy: 0.8653402328491211


In [79]:
#First boint in BONUS part to predict pased on user input 
def predfict_review(review, model, tokenizer, max_len):
    review = preprocess_text(review)
    sequence = tokenizer.texts_to_sequences([review])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    prediction = model.predict(padded_sequence)
    sentiment = np.argmax(prediction)
    sentiment_labels = {0: 'negative', 1: 'neutral', 2: 'positive'}
    return sentiment_labels[sentiment]
    


In [80]:
#Taking the input from the user and test using two model :Simple RNN and LSTM
new_review = input("Enter a new review: ")
rnn_sentiment = predfict_review(new_review, rnn_model, tokenizer, max_len)
lstm_sentiment = predfict_review(new_review, lstm_model, tokenizer, max_len)

print(f'RNN Model Prediction: {rnn_sentiment}')
print(f'LSTM Model Prediction: {lstm_sentiment}')


Enter a new review:  i like it


1/1 [==============================] - 0s 242ms/step
RNN Model Prediction: neutral
LSTM Model Prediction: neutral


In [81]:
#Second point in BONUS part 
def evaluate_and_train_for_LSTM_model(ratio_of_split, lengt_of_padding):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'], data['sentiments'], test_size=1-ratio_of_split, random_state=42)

    # Tokenization 
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)
    sequences_train = tokenizer.texts_to_sequences(X_train)
    sequences_test = tokenizer.texts_to_sequences(X_test)

    # Padding
    X_train_pad = pad_sequences(sequences_train, maxlen=padding_length)
    X_test_pad = pad_sequences(sequences_test, maxlen=padding_length)

    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Define and train LSTM model
    lstm_model = Sequential()
    lstm_model.add(Embedding(max_words, 128, input_length=padding_length))
    lstm_model.add(LSTM(128))
    lstm_model.add(Dense(3, activation='softmax'))
    lstm_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    lstm_model.fit(X_train_pad, y_train_cat, epochs=4, batch_size=64, validation_data=(X_test_pad, y_test_cat), verbose=0)

    # Evaluation of the model
    accuracy = lstm_model.evaluate(X_test_pad, y_test_cat, verbose=0)[1]
    
    return accuracy

ratios = [0.6, 0.7, 0.8]
lengths = [50, 100, 150]

answers = []

# Saving answers in dataframe
for split_ratio in ratios:
    for padding_length in lengths:
        accuracy = evaluate_and_train_for_LSTM_model(split_ratio, padding_length)
        answers.append({'Split Ratio': split_ratio, 'Padding Length': padding_length, 'Accuracy': accuracy})
answers_df = pd.DataFrame(answers)
best_hyperparameters = answers_df.loc[answers_df.groupby('Split Ratio')['Accuracy'].idxmax()]

print("LSTM Model best Hyperparameters:")
print(best_hyperparameters)
answers_df = answers_df.pivot(index='Split Ratio', columns='Padding Length', values='Accuracy')
answers_df.index.name = 'Split Ratio'
print("\nAccuracy against parameters:")
print(answers_df)
answers_df.to_csv('lstm_results.csv', index=False)



LSTM Model best Hyperparameters:
   Split Ratio  Padding Length  Accuracy
2          0.6             200  0.846864
3          0.7             100  0.860246
6          0.8             100  0.871972

Accuracy against parameters:
Padding Length       100       150       200
Split Ratio                                 
0.6             0.843980  0.846575  0.846864
0.7             0.860246  0.858324  0.856017
0.8             0.871972  0.864187  0.871972


In [82]:
#Second point in BONUS part 
def evaluate_and_train_for_simpleRNN_model(ratio_of_split, lengt_of_padding):
    # Split data
    X_train, X_test, y_train, y_test = train_test_split(data['cleaned_review'], data['sentiments'], test_size=1-ratio_of_split, random_state=42)

    # Tokenization 
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(X_train)
    sequences_train = tokenizer.texts_to_sequences(X_train)
    sequences_test = tokenizer.texts_to_sequences(X_test)

    # Padding
    X_train_pad = pad_sequences(sequences_train, maxlen=padding_length)
    X_test_pad = pad_sequences(sequences_test, maxlen=padding_length)

    y_train_cat = to_categorical(y_train, num_classes=3)
    y_test_cat = to_categorical(y_test, num_classes=3)

    # Train simple RNN model
    rnn_model = Sequential()
    rnn_model.add(Embedding(max_words, 128, input_length=padding_length))
    rnn_model.add(SimpleRNN(128))
    rnn_model.add(Dense(3, activation='softmax'))
    rnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    rnn_model.fit(X_train_pad, y_train_cat, epochs=4, batch_size=64, validation_data=(X_test_pad, y_test_cat), verbose=0)

    # Evaluation of the model
    accuracy = rnn_model.evaluate(X_test_pad, y_test_cat, verbose=0)[1]
    
    return accuracy

ratios = [0.6, 0.7, 0.8]
lengths = [50, 100, 150]

answers = []

# Saving answers in dataframe
for split_ratio in ratios:
    for padding_length in lengths:
        accuracy = evaluate_and_train_for_simpleRNN_model(split_ratio, padding_length)
        answers.append({'Split Ratio': split_ratio, 'Padding Length': padding_length, 'Accuracy': accuracy})
answers_df = pd.DataFrame(answers)
best_hyperparameters = answers_df.loc[answers_df.groupby('Split Ratio')['Accuracy'].idxmax()]

print("Simple RNN Model best Hyperparameters:")
print(best_hyperparameters)
answers_df = answers_df.pivot(index='Split Ratio', columns='Padding Length', values='Accuracy')
answers_df.index.name = 'Split Ratio'
print("\nAccuracy against parameters:")
print(answers_df)
answers_df.to_csv('rnn_results.csv', index=False)


Simple RNN Model best Hyperparameters:
   Split Ratio  Padding Length  Accuracy
0          0.6             100  0.839221
5          0.7             200  0.861207
6          0.8             100  0.856690

Accuracy against parameters:
Padding Length       100       150       200
Split Ratio                                 
0.6             0.839221  0.805768  0.725306
0.7             0.857555  0.828528  0.861207
0.8             0.856690  0.832180  0.687140
